In [ ]:
!python -V
!pip -V

## 1. Objetivos del notebook

Las APIs son aplicaciones muy populares que sirven de interfaz para intercambiar datos entre aplicaciones distintas. Este _notebook_ tiene por objetivos principal de familiarizarse con este concepto:
- desarrollando una API básica con Python y la librería Flask,
- describiendo algunos principios de buen diseño de API,
- construyendo una API para nuestra base de datos "Sophia2"
- utilizando la API de Twitter

## 2. ¿Qué es una API Web?

### 2.1 Definición

El termino API (__Application Programming Interface__) no es específico a las aplicaciones web. En informática, en general se refiere a una __parte de un programa diseñado para ser utilizado o manipulado por otro programa__ (ej: métodos públicos de una clase Java). A diferencia de una interfaz diseñada para ser utilizada o manipulada por un ser humano, los programas a menudo necesitan comunicarse entre sí o con el sistema operativo subyacente, y las API son una forma de hacerlo.

Una __API web__ permite que la información o funcionalidad de una aplicación sea manipulada por otros programas a través de Internet. Por ejemplo, con la API web de Twitter (https://developer.twitter.com/en/docs/twitter-api), puedes escribir un programa en un lenguaje como Python o Javascript que puede realizar tareas como favorecer tweets o recopilar metadatos de tweets.

### 2.2 ¿Cuándo crear una API Web?

En general, considere crear una API web si queremos compartir datos y:
- nuestros datos son grandes, lo que hace que la descarga sea difícil de manejar o requiera muchos recursos.
- nuestros datos cambian o se actualizan con frecuencia.

- nuestros usuarios necesitarán acceder a sus datos en tiempo real, por ejemplo para mostrarlos en otro sitio web o como parte de una aplicación.
- nuestros usuarios sólo necesitan acceder a una parte de los datos en un momento dado.
- nuestros usuarios tendrán que realizar acciones distintas a la recuperación de datos, como contribuir, actualizar o eliminar datos.

Si tiene datos que desea compartir con el mundo, una API es una forma de ponerlos en manos de otros. Sin embargo, las API no siempre son la mejor manera de compartir datos con los usuarios. Si el tamaño de los datos que está proporcionando es relativamente pequeño, puede proporcionar un __dump de datos__ en forma de archivo descargable JSON, XML, CSV o SQLite. Dependiendo de sus recursos, este enfoque puede ser viable hasta un tamaño de descarga de unos pocos gigabytes.

### 2.3 Conceptos asociados: _HTTP_, _URL_, _JSON_

Al utilizar o crear APIs, encontrará estos términos con frecuencia:

- <b>HTTP</b> (Hypertext Transfer Protocol) es un protocolo que implementa una serie de "métodos" que permite manipular recursos (datos o aplicaciones) de un servidor desde un computador cliente. Los dos más comunes son GET, que extrae datos de un servidor, y POST, que empuja nuevos datos a un servidor.
- <b>URL</b> (Uniform Resource Locator) - Una dirección para un recurso en la web.
- <b>JSON</b> (JavaScript Object Notation) es un formato de almacenamiento de datos basado en texto que está diseñado para ser fácil de leer tanto para humanos como para máquinas. JSON es generalmente el formato más común para devolver datos a través de una API, siendo XML el segundo más común.


### 2.4 ¿Que quieren los usuarios de una API? ¡Un buen diseño y una buena documentación!

La documentación es el punto de partida de un usuario cuando trabaja con una nueva API, y las URL bien diseñadas facilitan a los usuarios la búsqueda intuitiva de recursos. Dado que ayudan a los usuarios a acceder rápidamente a la información a través de su API, estos elementos (documentación y URL bien concebidas) son la condición sine qua non de una buena API.

A medida que utilice otras APIs en sus proyectos, desarrollará un sentido de lo que constituye una buena API desde la perspectiva de un usuario potencial. Del mismo modo que los lectores fuertes a menudo hacen escritores fuertes, el uso de APIs creadas por otros y la evaluación crítica de su implementación y documentación le ayudará a diseñar mejor sus propias APIs.

Ver APIs de Twitter: https://developer.twitter.com/en/docs.html

## 3. Desarrollar una API web básica con Python y Flask

### 3.1 Instalación de las librerías

Para este tutorial, necesitará Python 3 y la librería <i>Flask</i> (una librería Python que permite desarrollar aplicación web).

Una vez instalado <code>flask</code>, ejecutar el código siguiente para verificar que se instaló corectamente.

NB: Pueden ejecutar el código desde Jupyter Notebook (si lo tienen instalado) o un script Python que pueden llamar <code>server.py</code> por ejemplo.

In [ ]:
!pip install flask

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "<h1>¡Hola Mundo!</h1> <p>Bienvenido en nuestra aplicación web básica en Python</p>"

if __name__ == "__main__":
    app.run(debug=False,port=1234)

### 3.3 Creación de una primera API básica: Catalogo de libros de una librería

Vamos a desarrolla una API que propone compartir datos de los libros disponibles en una librería. Además del __título__ y la __fecha de publicación__, nuestra API también indicará la __primera frase__ de cada libro. Estos datos deberían ser suficientes para permitirnos visualizar algunas preguntas potenciales de investigación sin abrumarnos mientras nos enfocamos en el diseño de nuestra API. 

In [ ]:
import flask
from flask import request, jsonify

app = flask.Flask(__name__)

# Create some test data for our catalog in the form of a list of dictionaries.
books = [
    {'id': 0,
     'title': 'A Fire Upon the Deep',
     'author': 'Vernor Vinge',
     'first_sentence': 'The coldsleep itself was dreamless.',
     'year_published': '1992'},
    {'id': 1,
     'title': 'The Ones Who Walk Away From Omelas',
     'author': 'Ursula K. Le Guin',
     'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.',
     'published': '1973'},
    {'id': 2,
     'title': 'Dhalgren',
     'author': 'Samuel R. Delany',
     'first_sentence': 'to wound the autumnal city.',
     'published': '1975'}
]


@app.route('/', methods=['GET'])
def home():
    return "<h1>¡Hola Mundo!</h1> <p>Bienvenido en nuestra aplicación web básica en Python</p>"

# A route to return all of the available entries in our catalog.
@app.route('/api/v1/resources/books/all', methods=['GET'])
def api_all():
    return jsonify(books)

app.run(debug=False,port=1234)

Una vez que el servidor esté funcionando, visite nuestra URL con la ruta que permite ver los datos en el catálogo:

http://127.0.0.1:1234/api/v1/resources/books/all

Debería ver la salida JSON de las tres entradas en nuestro catálogo de pruebas. Flask nos proporciona una función <code>jsonify</code> que nos permite convertir listas y diccionarios al formato JSON. En la ruta que creamos, nuestras entradas de libro se convierten de una lista de diccionarios Python a JSON antes de ser devueltas a un usuario.

En este punto, ha creado una API que funciona, aunque limitada. En la siguiente sección, permitiremos a los usuarios encontrar libros a través de datos más específicos, como el ID de una entrada.

### 3.4 Encontrar recursos específicos

En este momento, los usuarios sólo pueden ver toda nuestra base de datos, no pueden filtrar ni encontrar recursos específicos. Si bien esto no es un problema con nuestro catálogo de pruebas, rápidamente se volverá menos útil a medida que agreguemos datos. En esta sección, añadiremos una función que permite a los usuarios filtrar los resultados devueltos mediante una petición más específica.

A continuación se muestra el código de nuestra nueva aplicación con capacidad de filtrado. Como antes, examinaremos el código más cuidadosamente una vez que lo tenga en funcionamiento.

In [ ]:
import flask
from flask import request, jsonify

app = flask.Flask(__name__)

# Create some test data for our catalog in the form of a list of dictionaries.
books = [
    {'id': 0,
     'title': 'A Fire Upon the Deep',
     'author': 'Vernor Vinge',
     'first_sentence': 'The coldsleep itself was dreamless.',
     'year_published': '1992'},
    {'id': 1,
     'title': 'The Ones Who Walk Away From Omelas',
     'author': 'Ursula K. Le Guin',
     'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.',
     'published': '1973'},
    {'id': 2,
     'title': 'Dhalgren',
     'author': 'Samuel R. Delany',
     'first_sentence': 'to wound the autumnal city.',
     'published': '1975'}
]


@app.route('/', methods=['GET'])
def home():
    return '''<h1>Nuestra API: Open Books</h1>
<p>Un prototipo de API para acceder a libros de ciencia ficción.</p>'''


@app.route('/api/v1/resources/books/all', methods=['GET'])
def api_all():
    return jsonify(books)


@app.route('/api/v1/resources/books', methods=['GET'])
def api_id():
    # Check if an ID was provided as part of the URL.
    # If ID is provided, assign it to a variable.
    # If no ID is provided, display an error in the browser.
    if 'id' in request.args:
        id = int(request.args['id'])
    else:
        return "Error: No id field provided. Please specify an id."

    # Create an empty list for our results
    results = []

    # Loop through the data and match results that fit the requested ID.
    # IDs are unique, but other fields might return many results
    for book in books:
        if book['id'] == id:
            results.append(book)

    # Use the jsonify function from Flask to convert our list of
    # Python dictionaries to the JSON format.
    return jsonify(results)


app.run(debug=False,port=1234)

Visite las siguientes URLs para probar la nueva capacidad de su API:

- http://127.0.0.1:1234/api/v1/resources/books?id=0
- http://127.0.0.1:1234/api/v1/resources/books?id=1
- http://127.0.0.1:1234/api/v1/resources/books?id=2
- http://127.0.0.1:1234/api/v1/resources/books?id=3

Cada uno de ellos debe devolver una entrada diferente, excepto el último, que debe devolver una lista vacía: [], ya que no hay ningún libro para el cual el valor de id sea 3. (El conteo en la programación típicamente comienza en 0, así que id=3 sería una petición para un cuarto ítem inexistente.

Examine la URL proporcionada para una identificación y seleccione los libros que coincidan con esa identificación. La identificación debe ser proporcionada de esta manera: ?id=0. Los datos pasados a través de URLs como ésta (después del ?) se llaman parámetros de consulta. Son una característica de HTTP utilizada para filtrar tipos específicos de datos.


## 4. Creación de una segunda API básica conectada al SGBD MongoDB

Vamos a crear una segunda API pero esta vez la API será conectada a Mongo para gestionar el almacenamiento de los datos.

La Base de Datos utilizada en esta sección es la base 'sophia2', una base de datos que hemos utizada en el tutorial anterior.

In [ ]:
!pip install flask_pymongo

In [ ]:
import flask
import json
from flask import request, jsonify

# Creación de una nueva aplicación web
app = flask.Flask(__name__)

# Definición de las rutas

@app.route('/', methods=['GET'])
def home():
    return '''<h1>API Sophia2</h1>
<p>Un prototipo de API para la base de datos Sophia2.</p>'''

@app.errorhandler(404)
def page_not_found(e):
    return "<h1>404</h1><p>The resource could not be found.</p>", 404

@app.route('/api_sophia2/v1/resources/news/all', methods=['GET'])
def api_all():
    ## Consultar Mongo para obtener datos sobre las noticias
    json_data=[]
    collection = db['news']
    for document in collection.find():
        json_data.append(document)
    return jsonify(json_data)

app.run(debug=False,port=1234)

In [ ]:
from flask import Flask
from flask import jsonify
from flask import request
from flask_pymongo import PyMongo

app = Flask(__name__)

app.config['MONGO_DBNAME'] = 'sophia2'
app.config['MONGO_URI'] = 'mongodb://localhost:27017/sophia2'

mongo = PyMongo(app)

@app.route('/api_sophia2/v1/resources/news/all', methods=['GET'])
def get_all_news():
    news = mongo.db.news
    output = []
    for n in news.find():
        output.append({'url' : n['url'], 'title' : n['title'], 'date': n['date']})
    return jsonify({'result' : output})


@app.route('/api_sophia2/v1/resources/news', methods=['GET'])
def get_specific_news_by_parameters():
    news = mongo.db.news
    output = []
    
    #parameters: "date" y "search"
    date = request.args['date']
    search = request.args['search']
        
    if (date=="null"):
        for n in news.find({'title': {'$regex' : ".*"+search+".*"}}):
            output.append({'url' : n['url'], 'title' : n['title'], 'date': n['date']})
    
    else:
        for n in news.find({'date':date, 'title': {'$regex' : ".*"+search+".*"}}):
            output.append({'url' : n['url'], 'title' : n['title'], 'date': n['date']})
    
    if len(output) == 0:
        return("No hay noticia en la fecha:"+date+" que tenga la palabra: "+search)
    
    return jsonify({'result' : output})

@app.route('/api_sophia2/v1/resources/news', methods=['POST'])
def add_news():
    news = mongo.db.news
    title = request.args['title']
    date = request.args['date']
    url = request.args['url']
    text = request.args['text']
    
    news_id = news.insert({'title': title, 'date': date, 'url': url, text:'text'})
    inserted_news = news.find_one({'_id': news_id })
  
    output = {'title' : inserted_news['title'], 'date' : inserted_news['date']}
    return jsonify({'result' : output})


app.run(debug=False,port=1234)

Para probar el método POST, copiar el código siguiente en script "cliente":

In [ ]:
import requests

params = {"title":"test", "url":"www", "text":"un texto", "date":"2021-06-23"}
headers = {'content-type': 'application/json'}

url = "http://127.0.0.1:1234/api_sophia2/v1/resources/news"

result = requests.post(url, params=params, headers=headers)

print(r.content)

## 5. Buenas prácticas de diseño de API

Antes de incluir más funcionalidad en nuestra aplicación, reflexionemos sobre algunas de las decisiones de diseño de API que hemos tomado hasta ahora. Dos aspectos de una buena API son la usabilidad y la mantenibilidad, y a medida que incorporemos más funcionalidad a nuestra API, tendremos en cuenta muchas de las siguientes consideraciones.

### 5.1 Definir acciones sobre los recursos con POST, GET, PUT y DELETE

La filosofía de diseño predominante de las API modernas se llama REST. Para nuestros propósitos, lo más importante de REST es que se basa en los cuatro métodos definidos por el protocolo HTTP: POST, GET, PUT y DELETE. Estas corresponden a las cuatro acciones tradicionales realizadas sobre los datos de una base de datos: CREAR, LEER, ACTUALIZAR y BORRAR. En este tutorial, sólo nos ocuparemos de las peticiones GET, que corresponden a la lectura de una base de datos.

### 5.2 Evitar incluir el tipo de acción en el URL

Consideremos un ejemplo débil o mal diseñado de un punto final de la API:

<code> http://api.example.com/getbook/10 </code>

El formato de esta consulta tiene varios problemas. El primero es semántico - en una API REST, nuestros verbos son típicamente GET, POST, PUT, o DELETE, y están determinados por el método de petición en lugar de por la URL de petición. Esto significa que la palabra "get" no debería aparecer en nuestra petición, ya que "get" está implícito en el hecho de que estamos usando un método HTTP GET. Además, las colecciones de recursos, como los libros o los usuarios, deben indicarse con sustantivos plurales. Esto deja claro cuando una API se refiere a una colección (libros) o una entrada (libro). Incorporando estos principios, nuestra API se vería así:

<code>http://api.example.com/books/10</code>

### 5.3 Evitar poner parametros en la ruta, mejor definirlos como parámetros

La consulta anterior utiliza parte de la ruta (/10) para proporcionar el ID. Aunque este no es un enfoque poco común, es un algo inflexible: con las URL construidas de esta manera, por lo general sólo se puede filtrar por un campo a la vez. Los parámetros de consulta permiten filtrar por múltiples campos de base de datos y tienen más sentido cuando se proporcionan datos "opcionales", como un formato de salida:

<code>http://api.example.com/books?author=Ursula+K.+LeGuin&published=1969&output=xml</code>

### 5.4 Planificar las adiciones futuras

Al diseñar cómo deben estructurarse las consultas a su API, también tiene sentido planificar las adiciones futuras. Incluso si la versión actual de su API sólo ofrece información sobre un tipo de recursos (por ejemplo, libros), tiene sentido planificar como si pudiera añadir otros recursos o funciones que no sean recursos a su API en el futuro:

<code>http://api.example.com/resources/books?id=10</code>

Agregar un segmento adicional en la ruta, como "recursos" o "entradas", le da la opción de permitir que los usuarios busquen en todos los recursos disponibles, lo que le facilita las solicitudes de soporte posteriores como éstas:

<code>https://api.example.com/v1/resources/images?id=10</code>

<code>https://api.example.com/v1/resources/all</code>

### 5.5 Definir versiones de su API

Otra forma de planificar el futuro de su API es añadir un número de versión a la ruta. Esto significa que, en caso de que tenga que rediseñar su API, puede seguir soportando la versión antigua de la API con el número de versión antiguo mientras libera, por ejemplo, una segunda versión (v2) con funcionalidad mejorada o diferente. De esta manera, las aplicaciones y los scripts creados utilizando la versión antigua de su API no dejarán de funcionar después de la actualización.

Después de incorporar estas mejoras de diseño, una solicitud a nuestra API podría verse así:

<code>https://api.example.com/v1/resources/books?id=10</code>

### 5.6 Inspirarse de otras API bien estructuradas y documentadas

Sin documentación, incluso la API mejor diseñada será inutilizable. Su API debe tener documentación que describa los recursos o la funcionalidad disponibles a través de su API que también proporcione ejemplos concretos de URL de solicitud o código para su API. Debe tener una sección para cada recurso que describa qué campos, como el id o el título, acepta. Cada sección debe tener un ejemplo en forma de una solicitud HTTP de ejemplo o un bloque de código.

Para obtener inspiración sobre cómo abordar la documentación de la API, consulte por ejemplo las API siguientes:

- <a href="http://api.repo.nypl.org/">New York Public Library Digital Collections API</a>
- <a href="https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation">World Bank API</a>
- <a href="https://developer.nytimes.com/">las APIS del New York Times</a>
- <a href="https://pro.europeana.eu/resources/apis">Europeana Pro API</a>
- <a href="https://developer.twitter.com/en/docs.html">APIs de Twitter</a>

## 6. Utilizar la API de Twitter 

documentación de Tweepy: https://docs.tweepy.org/en/stable/index.html

In [ ]:
!pip install tweepy

In [ ]:
import os
import tweepy as tw
import pandas as pd

In [ ]:
consumer_key= 'rMMtoLXxDB1FlazaN4A9CWZl7'
consumer_secret= 'BLYoo09azoLYes5lOOB0psa1urXIeucMDTpuZCyEo44GT4vgay'
access_token= '52061806-CJdzcwNG4S6LzerYqESmrVIxudyLOjFTnpGkeZXNC'
access_token_secret= 'Mtq4XM0SCeKY8iHoNb8L7z8vQ3XRsOZcdvZmKlmk1tdQT'

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# Define the search term and the date_since date as variables
search_words = "valdiviacl"
date_since = "2021-06-23"

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="es",
              tweet_mode="extended",
              since=date_since).items(5)
tweets

In [ ]:
# Iterate and print tweets
for tweet in tweets:
    print(tweet.full_text)

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words+"-filter:retweets",
              lang="es",
              tweet_mode="extended",
              since=date_since).items(50)
tweets

In [ ]:
# Iterate and print tweets
for tweet in tweets:
    print(tweet.full_text)

In [ ]:
users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
users_locs

In [ ]:
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user', "location"])
tweet_text

## 7. Preguntas/Ejercicios

- ¿Cuáles son los 'endpoints' de la API Twitter?
- ¿Cuáles son los recursos que provee la API de Twitter? Cuáles son su formato?
- ¿Cuáles son las buenas prácticas que siguen los desarrolladores de la API de Twitter?

- ¿A partir del modelo Entidad-Relación del caso de estudio "Sophia2", definir cuáles 'endpoints' se podrían definir?